In [11]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob
import spacy
from spacy.matcher import Matcher
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.probability import FreqDist
from collections import Counter
import re

In [72]:
tweet = pd.read_csv('tweetson04242020_time417pm.csv')


In [73]:
tweet['text_bkp'] = tweet['text']

In [74]:
tweet['text'] = tweet['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
tweet['text']

0        cityfalcon score gives top rated news for free...
1        no one thinks we’ve reached capitulation stage...
2        @jimcramer i had to turn you off today. you’re...
3        inside day scan results (stock > 10; volume > ...
4                                      hows $aapl looking?
                               ...                        
12508    the precision of alpha fibonacci targets on sp...
12509                 aapl: 239.01 at mar 24, 2020 5:55 pm
12510                 aapl: 239.01 at mar 24, 2020 5:55 pm
12511                 aapl: 239.01 at mar 24, 2020 5:55 pm
12512    coming back... $googl $msft $amzn $aapl $fbpic...
Name: text, Length: 12513, dtype: object

In [75]:
#Removing all hashtags, and twitter
tweet['text'] = tweet['text'].apply(lambda x: ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split()))

In [76]:
lem = WordNetLemmatizer()
tweet['text'] = tweet['text'].apply(lambda x: " ".join([lem.lemmatize(word) for word in x.split()]))

In [77]:
def sentiment(x):
    return TextBlob(x).sentiment  

tweet['score'] = tweet['text'].apply(sentiment)

In [78]:
tweet[['polarity', 'subjectivity']] = pd.DataFrame(tweet['score'].tolist(), index=tweet.index) 

In [79]:
tweet[['text','polarity','subjectivity']]

,text,polarity,subjectivity
0,cityfalcon score give top rated news for free ...,0.45,0.65
1,no one think we ve reached capitulation stage ...,0.00,0.00
2,i had to turn you off today you re misleading ...,0.20,0.60
3,inside day scan result stock 10 volume 200 00 ...,0.00,0.00
4,hows aapl looking,0.00,0.00
...,...,...,...
12508,the precision of alpha fibonacci target on sp ...,0.00,0.00
12509,aapl 239 01 at mar 24 2020 5 55 pm,0.00,0.00
12510,aapl 239 01 at mar 24 2020 5 55 pm,0.00,0.00
12511,aapl 239 01 at mar 24 2020 5 55 pm,0.00,0.00


In [80]:
def getVaderSentiment(text):
    sid_obj = SentimentIntensityAnalyzer() 
    sentiment_dict  = sid_obj.polarity_scores(text) 
    return sentiment_dict

In [81]:
tweet['vader_dict'] = tweet['text'].apply(getVaderSentiment)

In [82]:
tweet_sentiment = tweet['vader_dict'].apply(pd.Series)

In [83]:
result = pd.concat([tweet, tweet_sentiment], axis=1)
result.head()

,Unnamed: 0,text,timestamp,links,img_urls,username,text_bkp,score,polarity,subjectivity,vader_dict,neg,neu,pos,compound
0,0,cityfalcon score give top rated news for free ...,2020-03-30T23:59:48,['http://www.cityfalcon.com/watchlists?assets=...,['https://pbs.twimg.com/media/EUZVgw7WsAARSWK....,FinBuzz,CityFALCON score gives top rated news for FREE...,"(0.45, 0.65)",0.45,0.65,"{'neg': 0.0, 'neu': 0.718, 'pos': 0.282, 'comp...",0.000,0.718,0.282,0.6249
1,1,no one think we ve reached capitulation stage ...,2020-03-30T23:59:32,[],['https://pbs.twimg.com/media/EUZVccvXgAEe3x5....,PieterSta,No one thinks we’ve reached capitulation stage...,"(0.0, 0.0)",0.00,0.00,"{'neg': 0.102, 'neu': 0.833, 'pos': 0.065, 'co...",0.102,0.833,0.065,-0.2023
2,2,i had to turn you off today you re misleading ...,2020-03-30T23:58:14,[],[],Market Sucker,@jimcramer I had to turn you off today. You’re...,"(0.2, 0.6)",0.20,0.60,"{'neg': 0.069, 'neu': 0.867, 'pos': 0.064, 'co...",0.069,0.867,0.064,-0.0516
3,3,inside day scan result stock 10 volume 200 00 ...,2020-03-30T23:58:11,['https://www.finviz.com/screener.ashx?v=211&t...,[],Venky Srinivasan,Inside Day Scan Results (Stock > 10; Volume > ...,"(0.0, 0.0)",0.00,0.00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
4,4,hows aapl looking,2020-03-30T23:56:23,[],[],James B.,Hows $AAPL looking?,"(0.0, 0.0)",0.00,0.00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000


In [89]:
tweet_final = result[["timestamp","text_bkp","text","vader_dict","neg","neu","pos","compound"]]
tweet_final.head(25)
tweet_final.to_csv(r'tweet_sentiments.csv')